In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, names=rnames,  engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, names=mnames,  engine='python')
data = pd.merge(pd.merge(ratings, users), movies)

# Exercises: 

**MovieLens database**

1- Filter  films that have received at least 250 ratings. **Use only these films in the rest of exercices**

In [2]:
def ex1():
    # Gets the number of ratings received by each movie.
    ratingsPerMovie = data.groupby('movie_id').size()
    
    # Keeps only the movies with more than 249 ratings. (Keep into mind that index = movie_id, value = number of ratings)
    enoughRatings = ratingsPerMovie[ratingsPerMovie >= 250]

    # Keeps only the interactions that include one of the movies with enough ratings. Group by Title and sort descending in order to show in the same format as result spreadsheet.
    result = data[data['movie_id'].isin(enoughRatings.index)]
    
    # Show the result in the same way as it is shown in the result spreadsheet.
    return result.groupby('title').size().sort_values(ascending=False)

ex1()

title
American Beauty (1999)                                            3428
Star Wars: Episode IV - A New Hope (1977)                         2991
Star Wars: Episode V - The Empire Strikes Back (1980)             2990
Star Wars: Episode VI - Return of the Jedi (1983)                 2883
Jurassic Park (1993)                                              2672
                                                                  ... 
Wes Craven's New Nightmare (1994)                                  251
Teenage Mutant Ninja Turtles II: The Secret of the Ooze (1991)     251
Top Hat (1935)                                                     251
Random Hearts (1999)                                               250
U Turn (1997)                                                      250
Length: 1216, dtype: int64

2- Obtain the mean ratings for each movie grouped by gender that have at least 250 ratings. 

In [3]:
def ex2():
    # Keep only relevant movies using ex1()
    relevantMovies = data[data['title'].isin(ex1().index)]

    # Group all interactions by title and gender, applying mean operation on rating column
    relevantMoviesUngrouped = relevantMovies.groupby(['title', 'gender'], as_index=False)['rating'].mean() 

    # Show results as spreadsheets expects them to be
    auxM = relevantMoviesUngrouped[relevantMoviesUngrouped['gender'] == 'F'].reset_index(drop = True).drop(['gender'], axis = 1)
    auxF = relevantMoviesUngrouped[relevantMoviesUngrouped['gender'] == 'M'].reset_index(drop = True).drop(['gender'], axis = 1)
    auxF.columns = ['delete', 'rating']

    result = pd.concat([auxM, auxF], axis=1).drop(['delete'], axis=1)
    return result.set_index('title')

ex2()

,rating,rating
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


3- Show films more valued by women.

In [4]:
# What results show
def ex3():
    # Keep only relevant movies using ex1()
    relevantMovies = data[data['title'].isin(ex1().index)]
    
    # Keep only interactions by women
    relevantFMovies = relevantMovies[relevantMovies['gender'] == 'F']

    # Group all interactions by title, applying mean operation on rating and sorting from top to bottom
    return relevantFMovies.groupby(['title'])['rating'].mean().sort_values(ascending=False)

ex3()

title
Close Shave, A (1995)                                     4.644444
Wrong Trousers, The (1993)                                4.588235
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)             4.572650
Wallace & Gromit: The Best of Aardman Animation (1996)    4.563107
Schindler's List (1993)                                   4.562602
                                                            ...   
Avengers, The (1998)                                      1.915254
Speed 2: Cruise Control (1997)                            1.906667
Rocky V (1990)                                            1.878788
Barb Wire (1996)                                          1.585366
Battlefield Earth (2000)                                  1.574468
Name: rating, Length: 1216, dtype: float64

4- Now we wonder which movies are rated more differently between men and women. Which films have more different rating and are more highly valued by women? 

In [5]:
def ex4():
    # Keep only relevant movies using ex1()
    relevantMovies = data[data['title'].isin(ex1().index)]

    # We create a dataframe with both ratings by M and F in seperate columns
    f = relevantMovies[relevantMovies['gender'] == 'F'].groupby(['title', 'gender'])['rating'].mean().reset_index(level=1, drop=True)
    m = relevantMovies[relevantMovies['gender'] == 'M'].groupby(['title', 'gender'])['rating'].mean().reset_index(level=1, drop=True)
    genderRatings = pd.concat([f, m], axis = 1)
    genderRatings.columns = ["F", "M"]

    # We add a column to the frame that will contain the difference between the ratings by F and M
    genderRatings["diff"] = genderRatings["M"] - genderRatings["F"]

    # We show the results as expected by the spreadsheet
    return genderRatings.sort_values(by = "diff")

ex4()

,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
...,...,...,...
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
"Longest Day, The (1962)",3.411765,4.031447,0.619682
Dumb & Dumber (1994),2.697987,3.336595,0.638608


5- And which films have more different rating and are more highly valued by men? 

In [6]:
def ex5():
    # We get the calculations from ex4() and just reverse them
    toBeReversed = ex4()
    
    return toBeReversed[['F', 'M', 'diff']].sort_values(by = "diff", ascending = False)

ex5()

,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
...,...,...,...
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Little Women (1994),3.870588,3.321739,-0.548849
Grease (1978),3.975265,3.367041,-0.608224


6- What are the films that have generated the most discordant ratings, regardless of gender?

In [7]:
def ex6():
    # Keep only relevant movies using ex1()
    relevantMovies = data[data['title'].isin(ex1().index)]
    
    # Group all interactions by title, applying mean operation on rating and sorting from top to bottom
    return relevantMovies.groupby(['title'])['rating'].std().sort_values(ascending=False)
    
ex6()

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
                                           ...   
Wrong Trousers, The (1993)               0.708666
Shawshank Redemption, The (1994)         0.700443
Great Escape, The (1963)                 0.692585
Rear Window (1954)                       0.688946
Close Shave, A (1995)                    0.667143
Name: rating, Length: 1216, dtype: float64

7- What is the highest rated movie in average?

In [8]:
def ex7():
    # Keep only relevant movies using ex1()
    relevantMovies = data[data['title'].isin(ex1().index)]
    
    # Group all interactions by title, applying mean operation on rating and sorting from top to bottom. Then, keep only the first movie, as it will be the highest rated one.
    return relevantMovies.groupby(['title'])['rating'].mean().sort_values(ascending=False).head(1)

ex7()

title
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)    4.56051
Name: rating, dtype: float64

*** From here use all the ratings ***

8- Calculate the average rating of each user. 

In [9]:
def ex8():
    # We group all ratings by user_id and calculate the mean for each user.
    return data.groupby(['user_id'])['rating'].mean()

ex8()

user_id
1       4.188679
2       3.713178
3       3.901961
4       4.190476
5       3.146465
          ...   
6036    3.302928
6037    3.717822
6038    3.800000
6039    3.878049
6040    3.577713
Name: rating, Length: 6040, dtype: float64

9- Define a function called  <b>top_movies</b> that given a df, an user an a number n it returns what movies have the top n rank for this user.

def top_movies(data,user,n)


In [10]:
def top_movies(data_,usr,n_rows = 10):
    # We filter the iteractions and keep those done by the given user, order by rating (top to bottom) and show only the n best movies
    result = data[data['user_id'] == usr].sort_values(by = 'rating', ascending=False).head(n_rows)
    # We show the result as expected by the result spreadsheet
    return result[['title', 'rating']]
def ex9():
    return top_movies(data,1,30)
    
ex9()

,title,rating
0,One Flew Over the Cuckoo's Nest (1975),5
50759,Dumbo (1941),5
41626,Toy Story (1995),5
19503,Awakenings (1990),5
43703,Rain Man (1988),5
25853,Schindler's List (1993),5
37339,Cinderella (1950),5
15859,"Sound of Music, The (1965)",5
28501,Pocahontas (1995),5
49748,Mary Poppins (1964),5


** Data from CSV**

10- Read data from csv file: `ma-ba.csv`. Count the number of times `Barça` wins `Madrid` and compute the stadistics of % win, % lose and % draw.

In [11]:
def ex10():
    # We read data from the file
    matches = pd.read_csv('ma-ba.csv', sep=',', header=0, names = ["#", "Fecha", "Lugar", "Partido", "Resultado", "Competicion"], engine='python')
    
    # We split the "Resultado" column for more information
    matches[['Local', 'Visitante']] = matches["Resultado"].str.split("-",expand=True,)

    # Now we separate those matches in which Barça is the home team from those in which it is not
    # Barça is the home team if 'B' shows before than 'M' in the "Partido" column value (no possible wording of Barça's name includes an 'M', while no possible wording of Madrid's name includes a 'B')
    homeBarcaMatches = matches[matches["Partido"].str.upper().str.find('B') < matches["Partido"].str.upper().str.find('M')]
    awayBarcaMatches = matches[matches["Partido"].str.upper().str.find('B') > matches["Partido"].str.upper().str.find('M')]

    # We change "Local" and "Visitante" column names for the corresponding team name
    homeBarcaMatchesRenamed = homeBarcaMatches[["Local", "Visitante"]]
    homeBarcaMatchesRenamed.columns = ["FCB", "RM"]
    awayBarcaMatchesRenamed = awayBarcaMatches[["Local", "Visitante"]]
    awayBarcaMatchesRenamed.columns = ["RM", "FCB"]

    # We put all the matches together in unified format
    matchesRenamed = pd.concat([homeBarcaMatchesRenamed, awayBarcaMatchesRenamed])

    # We create the desired result dataframe, as requested by the result spreadsheet, based on the information recollected in matchesRenamed
    barcelonaWins = matchesRenamed[matchesRenamed["FCB"] > matchesRenamed["RM"]].index.size
    draws = matchesRenamed[matchesRenamed["FCB"] == matchesRenamed["RM"]].index.size
    madridWins = matchesRenamed[matchesRenamed["FCB"] < matchesRenamed["RM"]].index.size
    totalMatches = matches.index.size

    resultBase = {
        'team': ["Barcelona", "Madrid"],
        'wins': [barcelonaWins, madridWins],
        'losses': [madridWins, barcelonaWins],
        'draws': [draws, draws], 
        '%win': [barcelonaWins/totalMatches, madridWins/totalMatches],
        '%lose': [madridWins/totalMatches, barcelonaWins/totalMatches],
        '%draw': [draws/totalMatches, draws/totalMatches]
    }

    results = pd.DataFrame(resultBase, columns=['team', 'wins','losses','draws','%win','%lose', '%draw']) 
    results.set_index('team', inplace=True)
    return results

ex10()

,wins,losses,draws,%win,%lose,%draw
team,,,,,,
Barcelona,107,94,59,0.411538,0.361538,0.226923
Madrid,94,107,59,0.361538,0.411538,0.226923


In [116]:
n = 11
for i in range(1,n):
    try:
        df = globals()["ex"+str(i)]()
        df.to_csv(str(i)+".csv",header=False)
    except Exception as e: 
        print(i,repr(e))
        open(str(i)+".csv","a").close()
